# 导入相关库

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

# 读数据

In [2]:
data = pd.read_excel('./2023年中国研究生数学建模竞赛赛题/E题/data3a.xlsx')
data_image = pd.read_excel('./2023年中国研究生数学建模竞赛赛题/E题/竞赛发布数据/表3-患者影像信息血肿及水肿的形状及灰度分布.xlsx')


In [3]:
data = data.rename(columns={'入院首次影像检查流水号':'流水号'})

In [4]:
data = data.merge(data_image, on='流水号', how='left')

In [5]:
train = data.iloc[:100, :]
test = data.iloc[100:, :]

In [6]:
label = train['90天mRS']

In [7]:
label.value_counts()

3.0    20
2.0    20
1.0    19
5.0    15
4.0    12
0.0    10
6.0     4
Name: 90天mRS, dtype: int64

# 特征工程

In [8]:
train.drop(['ID', '流水号', '90天mRS'], axis=1, inplace=True)
test.drop(['ID', '流水号'], axis=1, inplace=True)
train['高压'] = train['血压'].apply(lambda x: int(x.split('/')[0]))
train['低压'] = train['血压'].apply(lambda x: int(x.split('/')[1]))
test['高压'] = test['血压'].apply(lambda x: int(x.split('/')[0]))
test['低压'] = test['血压'].apply(lambda x: int(x.split('/')[1]))
train.drop(['血压'], axis=1, inplace=True)
test.drop(['血压'], axis=1, inplace=True)
train['性别'] = train['性别'].map({'男':0, '女':1})
test['性别'] = test['性别'].map({'男':0, '女':1})
feat = train.columns
scaler = StandardScaler()
train[feat] = scaler.fit_transform(train[feat])
test[feat] = scaler.transform(test[feat])
print(len(feat))

73


In [9]:
feat = ['糖尿病史',
 'HM_Cerebellum_R_Ratio',
 'original_shape_Maximum2DDiameterSlice',
 'NCCT_original_firstorder_InterquartileRange',
 'original_shape_Elongation']

# 交叉验证

In [10]:
# oof = np.zeros(len(train))
# feat_imp = np.zeros(len(feat))
# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# for i, (tra_index, val_index) in enumerate(skf.split(train, label)):
#     print(f'第{i+1}折....')
#     tra_x = train.iloc[tra_index, :][feat].values
#     tra_y = label.iloc[tra_index].values
#     val_x = train.iloc[val_index, :][feat].values
#     val_y = label.iloc[val_index].values
#     model = LogisticRegression(random_state=42)
#     model.fit(tra_x, tra_y)
#     feat_imp += abs(np.mean(model.coef_, axis=0))
#     pred_soft = model.predict_proba(val_x)[:, 1]
#     pred = model.predict(val_x)
#     score = accuracy_score(val_y, pred)
#     print(f'准确率为{score}')
#     oof[val_index] = pred
# oof_score = accuracy_score(label.values, oof)
# feat_imp /= 5
# print('*' * 30)
# print(f'OOF 准确率 = {oof_score}')

In [11]:
oof = np.zeros(len(train))
feat_imp = np.zeros(len(feat))
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
params = {'booster': 'gbtree', 
         'device': 'cpu', 
         'eta': 0.01, 
         'max_depth': 6, 
         'objective': 'multi:softmax',
          'num_round': 300, 
         'seed': 42, 
         'num_class': 7}
for i, (tra_index, val_index) in enumerate(skf.split(train, label)):
    print(f'第{i+1}折....')
    tra_x = train.iloc[tra_index, :][feat]
    tra_y = label.iloc[tra_index].values
    val_x = train.iloc[val_index, :][feat]
    val_y = label.iloc[val_index].values
    model = xgb.XGBClassifier(params, early_stopping_rounds=30)
    model.fit(tra_x, tra_y, eval_set=[(val_x, val_y)])
    feat_imp += model.feature_importances_
#    pred_soft = model.predict_proba(val_x)[:, 1]
    pred = model.predict(val_x)
    score = accuracy_score(val_y, pred)
    print(f'准确率为{score}')
    oof[val_index] = pred
oof_score = accuracy_score(label.values, oof)
feat_imp /= 5
print('*' * 30)
print(f'OOF 准确率 = {oof_score}')

第1折....
[0]	validation_0-mlogloss:1.81535
[1]	validation_0-mlogloss:1.79290
[2]	validation_0-mlogloss:1.73747
[3]	validation_0-mlogloss:1.73692
[4]	validation_0-mlogloss:1.71227
[5]	validation_0-mlogloss:1.70569
[6]	validation_0-mlogloss:1.69726
[7]	validation_0-mlogloss:1.72364
[8]	validation_0-mlogloss:1.73545
[9]	validation_0-mlogloss:1.73382
[10]	validation_0-mlogloss:1.75742
[11]	validation_0-mlogloss:1.77970
[12]	validation_0-mlogloss:1.80583
[13]	validation_0-mlogloss:1.80179
[14]	validation_0-mlogloss:1.79839
[15]	validation_0-mlogloss:1.83315
[16]	validation_0-mlogloss:1.85457
[17]	validation_0-mlogloss:1.86558
[18]	validation_0-mlogloss:1.89756
[19]	validation_0-mlogloss:1.91264
[20]	validation_0-mlogloss:1.93313
[21]	validation_0-mlogloss:1.94658
[22]	validation_0-mlogloss:1.96974
[23]	validation_0-mlogloss:1.97319
[24]	validation_0-mlogloss:1.98957
[25]	validation_0-mlogloss:1.99987
[26]	validation_0-mlogloss:2.00511
[27]	validation_0-mlogloss:2.00840
[28]	validation_0-mlog

# 特征选择

In [12]:
# feature_import = pd.DataFrame()
# feature_import['columns'] = feat
# feature_import['score'] = feat_imp
# feature_import = feature_import.sort_values(by='score', ascending=False)
# best_thre = 0
# best_score = 0
# for j in np.arange(0, 0.03, 0.001):
#     print(f'阈值={j}')
#     feat = list(feature_import[feature_import['score'] >= j]['columns'].values)
#     oof = np.zeros(len(train))
#     skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
#     for i, (tra_index, val_index) in enumerate(skf.split(train, label)):
#         tra_x = train.iloc[tra_index, :][feat]
#         tra_y = label.iloc[tra_index].values
#         val_x = train.iloc[val_index, :][feat]
#         val_y = label.iloc[val_index].values
#         model = xgb.XGBClassifier(params, early_stopping_rounds=30)
#         model.fit(tra_x, tra_y, eval_set=[(val_x, val_y)])
#         pred = model.predict(val_x)
#         score = accuracy_score(val_y, pred)
#         oof[val_index] = pred
#     oof_score = accuracy_score(label.values, oof)
#     if oof_score > best_score:
#         best_score = oof_score
#         best_thre = j
#     print(f'OOF 准确率 = {oof_score}')
#     print('*' * 30)

# print(list(feature_import[feature_import['score'] >= best_thre]['columns'].values))
# print(best_score)

阈值=0.0
[0]	validation_0-mlogloss:1.97617
[1]	validation_0-mlogloss:1.91841
[2]	validation_0-mlogloss:1.91369
[3]	validation_0-mlogloss:1.92320
[4]	validation_0-mlogloss:1.92523
[5]	validation_0-mlogloss:1.92560
[6]	validation_0-mlogloss:1.94187
[7]	validation_0-mlogloss:1.96023
[8]	validation_0-mlogloss:1.99435
[9]	validation_0-mlogloss:1.97358
[10]	validation_0-mlogloss:1.99358
[11]	validation_0-mlogloss:1.97582
[12]	validation_0-mlogloss:1.99040
[13]	validation_0-mlogloss:2.00245
[14]	validation_0-mlogloss:2.00664
[15]	validation_0-mlogloss:2.00094
[16]	validation_0-mlogloss:1.98667
[17]	validation_0-mlogloss:1.99640
[18]	validation_0-mlogloss:2.00854
[19]	validation_0-mlogloss:2.00541
[20]	validation_0-mlogloss:2.01620
[21]	validation_0-mlogloss:2.00464
[22]	validation_0-mlogloss:2.01223
[23]	validation_0-mlogloss:2.00655
[24]	validation_0-mlogloss:1.99207
[25]	validation_0-mlogloss:2.00805
[26]	validation_0-mlogloss:2.00272
[27]	validation_0-mlogloss:2.00976
[28]	validation_0-mlogl

[14]	validation_0-mlogloss:2.01851
[15]	validation_0-mlogloss:2.04725
[16]	validation_0-mlogloss:2.07294
[17]	validation_0-mlogloss:2.08610
[18]	validation_0-mlogloss:2.10230
[19]	validation_0-mlogloss:2.12573
[20]	validation_0-mlogloss:2.12852
[21]	validation_0-mlogloss:2.13167
[22]	validation_0-mlogloss:2.12408
[23]	validation_0-mlogloss:2.13175
[24]	validation_0-mlogloss:2.13083
[25]	validation_0-mlogloss:2.12383
[26]	validation_0-mlogloss:2.14464
[27]	validation_0-mlogloss:2.15624
[28]	validation_0-mlogloss:2.17730
[29]	validation_0-mlogloss:2.19106
[30]	validation_0-mlogloss:2.19325
[31]	validation_0-mlogloss:2.19423
[0]	validation_0-mlogloss:1.88566
[1]	validation_0-mlogloss:1.87298
[2]	validation_0-mlogloss:1.90402
[3]	validation_0-mlogloss:1.89501
[4]	validation_0-mlogloss:1.92653
[5]	validation_0-mlogloss:1.97464
[6]	validation_0-mlogloss:2.01331
[7]	validation_0-mlogloss:2.05563
[8]	validation_0-mlogloss:2.09072
[9]	validation_0-mlogloss:2.12264
[10]	validation_0-mlogloss:2.1

[29]	validation_0-mlogloss:2.40924
[30]	validation_0-mlogloss:2.41667
[31]	validation_0-mlogloss:2.41393
OOF 准确率 = 0.29
******************************
阈值=0.003
[0]	validation_0-mlogloss:1.97617
[1]	validation_0-mlogloss:1.91841
[2]	validation_0-mlogloss:1.91369
[3]	validation_0-mlogloss:1.92320
[4]	validation_0-mlogloss:1.92523
[5]	validation_0-mlogloss:1.92560
[6]	validation_0-mlogloss:1.94187
[7]	validation_0-mlogloss:1.96023
[8]	validation_0-mlogloss:1.99435
[9]	validation_0-mlogloss:1.97358
[10]	validation_0-mlogloss:1.99358
[11]	validation_0-mlogloss:1.97582
[12]	validation_0-mlogloss:1.99040
[13]	validation_0-mlogloss:2.00245
[14]	validation_0-mlogloss:2.00664
[15]	validation_0-mlogloss:2.00094
[16]	validation_0-mlogloss:1.98667
[17]	validation_0-mlogloss:1.99640
[18]	validation_0-mlogloss:2.00854
[19]	validation_0-mlogloss:2.00541
[20]	validation_0-mlogloss:2.01620
[21]	validation_0-mlogloss:2.00464
[22]	validation_0-mlogloss:2.01223
[23]	validation_0-mlogloss:2.00655
[24]	valid

[11]	validation_0-mlogloss:1.95942
[12]	validation_0-mlogloss:1.97060
[13]	validation_0-mlogloss:1.99422
[14]	validation_0-mlogloss:2.01851
[15]	validation_0-mlogloss:2.04725
[16]	validation_0-mlogloss:2.07294
[17]	validation_0-mlogloss:2.08610
[18]	validation_0-mlogloss:2.10230
[19]	validation_0-mlogloss:2.12573
[20]	validation_0-mlogloss:2.12852
[21]	validation_0-mlogloss:2.13764
[22]	validation_0-mlogloss:2.13681
[23]	validation_0-mlogloss:2.13397
[24]	validation_0-mlogloss:2.12020
[25]	validation_0-mlogloss:2.12821
[26]	validation_0-mlogloss:2.14821
[27]	validation_0-mlogloss:2.17190
[28]	validation_0-mlogloss:2.17612
[29]	validation_0-mlogloss:2.18668
[30]	validation_0-mlogloss:2.19192
[31]	validation_0-mlogloss:2.19674
[32]	validation_0-mlogloss:2.19778
[0]	validation_0-mlogloss:1.88566
[1]	validation_0-mlogloss:1.87298
[2]	validation_0-mlogloss:1.90402
[3]	validation_0-mlogloss:1.89501
[4]	validation_0-mlogloss:1.92653
[5]	validation_0-mlogloss:1.97464
[6]	validation_0-mlogloss:

[22]	validation_0-mlogloss:2.34994
[23]	validation_0-mlogloss:2.35784
[24]	validation_0-mlogloss:2.35988
[25]	validation_0-mlogloss:2.36895
[26]	validation_0-mlogloss:2.37941
[27]	validation_0-mlogloss:2.38908
[28]	validation_0-mlogloss:2.40190
[29]	validation_0-mlogloss:2.40924
[30]	validation_0-mlogloss:2.41667
OOF 准确率 = 0.29
******************************
阈值=0.006
[0]	validation_0-mlogloss:1.97617
[1]	validation_0-mlogloss:1.91841
[2]	validation_0-mlogloss:1.91369
[3]	validation_0-mlogloss:1.92320
[4]	validation_0-mlogloss:1.92523
[5]	validation_0-mlogloss:1.92560
[6]	validation_0-mlogloss:1.94187
[7]	validation_0-mlogloss:1.95450
[8]	validation_0-mlogloss:1.96671
[9]	validation_0-mlogloss:1.97315
[10]	validation_0-mlogloss:1.99585
[11]	validation_0-mlogloss:2.01412
[12]	validation_0-mlogloss:2.01348
[13]	validation_0-mlogloss:1.99954
[14]	validation_0-mlogloss:2.00510
[15]	validation_0-mlogloss:2.03860
[16]	validation_0-mlogloss:2.05567
[17]	validation_0-mlogloss:2.04660
[18]	valid

...

[23]	validation_0-mlogloss:1.84567
[24]	validation_0-mlogloss:1.86805
[25]	validation_0-mlogloss:1.87208
[26]	validation_0-mlogloss:1.90098
[27]	validation_0-mlogloss:1.90490
[28]	validation_0-mlogloss:1.92447
[29]	validation_0-mlogloss:1.93135
[30]	validation_0-mlogloss:1.93645
[31]	validation_0-mlogloss:1.94799
[32]	validation_0-mlogloss:1.94837
[33]	validation_0-mlogloss:1.96645
[34]	validation_0-mlogloss:1.97445
[35]	validation_0-mlogloss:1.97748
[36]	validation_0-mlogloss:1.99087
[0]	validation_0-mlogloss:1.85376
[1]	validation_0-mlogloss:1.81056
[2]	validation_0-mlogloss:1.76110
[3]	validation_0-mlogloss:1.75679
[4]	validation_0-mlogloss:1.77394
[5]	validation_0-mlogloss:1.79816
[6]	validation_0-mlogloss:1.77295
[7]	validation_0-mlogloss:1.80453
[8]	validation_0-mlogloss:1.78827
[9]	validation_0-mlogloss:1.80567
[10]	validation_0-mlogloss:1.84033
[11]	validation_0-mlogloss:1.86092
[12]	validation_0-mlogloss:1.89369
[13]	validation_0-mlogloss:1.92660
[14]	validation_0-mlogloss:1.9

[11]	validation_0-mlogloss:1.92964
[12]	validation_0-mlogloss:1.96067
[13]	validation_0-mlogloss:1.97090
[14]	validation_0-mlogloss:2.00709
[15]	validation_0-mlogloss:2.03475
[16]	validation_0-mlogloss:2.08058
[17]	validation_0-mlogloss:2.10308
[18]	validation_0-mlogloss:2.12709
[19]	validation_0-mlogloss:2.17500
[20]	validation_0-mlogloss:2.20889
[21]	validation_0-mlogloss:2.22359
[22]	validation_0-mlogloss:2.24324
[23]	validation_0-mlogloss:2.26258
[24]	validation_0-mlogloss:2.29064
[25]	validation_0-mlogloss:2.29055
[26]	validation_0-mlogloss:2.30793
[27]	validation_0-mlogloss:2.31773
[28]	validation_0-mlogloss:2.32797
[29]	validation_0-mlogloss:2.32796
[30]	validation_0-mlogloss:2.33718
[31]	validation_0-mlogloss:2.34084
[32]	validation_0-mlogloss:2.35807
[0]	validation_0-mlogloss:1.90124
[1]	validation_0-mlogloss:1.86894
[2]	validation_0-mlogloss:1.88122
[3]	validation_0-mlogloss:1.85599
[4]	validation_0-mlogloss:1.87605
[5]	validation_0-mlogloss:1.88971
[6]	validation_0-mlogloss:

[0]	validation_0-mlogloss:1.85273
[1]	validation_0-mlogloss:1.79037
[2]	validation_0-mlogloss:1.75403
[3]	validation_0-mlogloss:1.68402
[4]	validation_0-mlogloss:1.63919
[5]	validation_0-mlogloss:1.60125
[6]	validation_0-mlogloss:1.61950
[7]	validation_0-mlogloss:1.62109
[8]	validation_0-mlogloss:1.64429
[9]	validation_0-mlogloss:1.64297
[10]	validation_0-mlogloss:1.67289
[11]	validation_0-mlogloss:1.67592
[12]	validation_0-mlogloss:1.70108
[13]	validation_0-mlogloss:1.69197
[14]	validation_0-mlogloss:1.70921
[15]	validation_0-mlogloss:1.71391
[16]	validation_0-mlogloss:1.73189
[17]	validation_0-mlogloss:1.73889
[18]	validation_0-mlogloss:1.75833
[19]	validation_0-mlogloss:1.75483
[20]	validation_0-mlogloss:1.76645
[21]	validation_0-mlogloss:1.77136
[22]	validation_0-mlogloss:1.78555
[23]	validation_0-mlogloss:1.79450
[24]	validation_0-mlogloss:1.80493
[25]	validation_0-mlogloss:1.82748
[26]	validation_0-mlogloss:1.83052
[27]	validation_0-mlogloss:1.85408
[28]	validation_0-mlogloss:1.8

[20]	validation_0-mlogloss:2.54613
[21]	validation_0-mlogloss:2.53762
[22]	validation_0-mlogloss:2.57758
[23]	validation_0-mlogloss:2.59011
[24]	validation_0-mlogloss:2.61603
[25]	validation_0-mlogloss:2.62190
[26]	validation_0-mlogloss:2.65093
[27]	validation_0-mlogloss:2.66083
[28]	validation_0-mlogloss:2.67579
[29]	validation_0-mlogloss:2.69820
[0]	validation_0-mlogloss:1.90613
[1]	validation_0-mlogloss:1.87437
[2]	validation_0-mlogloss:1.86853
[3]	validation_0-mlogloss:1.87246
[4]	validation_0-mlogloss:1.87898
[5]	validation_0-mlogloss:1.91749
[6]	validation_0-mlogloss:1.93469
[7]	validation_0-mlogloss:1.94630
[8]	validation_0-mlogloss:1.96808
[9]	validation_0-mlogloss:1.99643
[10]	validation_0-mlogloss:2.00461
[11]	validation_0-mlogloss:2.02938
[12]	validation_0-mlogloss:2.04822
[13]	validation_0-mlogloss:2.04631
[14]	validation_0-mlogloss:2.07302
[15]	validation_0-mlogloss:2.07039
[16]	validation_0-mlogloss:2.06961
[17]	validation_0-mlogloss:2.10699
[18]	validation_0-mlogloss:2.1

[30]	validation_0-mlogloss:2.66413
[0]	validation_0-mlogloss:1.88833
[1]	validation_0-mlogloss:1.86867
[2]	validation_0-mlogloss:1.85121
[3]	validation_0-mlogloss:1.84584
[4]	validation_0-mlogloss:1.84029
[5]	validation_0-mlogloss:1.86529
[6]	validation_0-mlogloss:1.88305
[7]	validation_0-mlogloss:1.89990
[8]	validation_0-mlogloss:1.92751
[9]	validation_0-mlogloss:1.96090
[10]	validation_0-mlogloss:2.01542
[11]	validation_0-mlogloss:2.04326
[12]	validation_0-mlogloss:2.08120
[13]	validation_0-mlogloss:2.11254
[14]	validation_0-mlogloss:2.15577
[15]	validation_0-mlogloss:2.19286
[16]	validation_0-mlogloss:2.21323
[17]	validation_0-mlogloss:2.24552
[18]	validation_0-mlogloss:2.29503
[19]	validation_0-mlogloss:2.33776
[20]	validation_0-mlogloss:2.34896
[21]	validation_0-mlogloss:2.37733
[22]	validation_0-mlogloss:2.41122
[23]	validation_0-mlogloss:2.41488
[24]	validation_0-mlogloss:2.42972
[25]	validation_0-mlogloss:2.44279
[26]	validation_0-mlogloss:2.46659
[27]	validation_0-mlogloss:2.4

# 全数据训练和预测

In [15]:
params = {'booster': 'gbtree', 
         'device': 'cpu', 
         'eta': 0.01, 
         'max_depth': 6, 
         'objective': 'multi:softmax',
          'num_round': 5, 
         'seed': 42, 
         'num_class': 7}
model = xgb.XGBClassifier(params)
model.fit(tra_x, tra_y)
train_pred = model.predict(train[feat])
test_pred = model.predict(test[feat])
all_pred = np.concatenate([train_pred, test_pred])
data['pred'] = all_pred
data[['ID', 'pred']].to_csv('3a.csv', index=None)

In [14]:
#可改进的地方：特征工程，使用多个模型，特征选择，调参，多模型集成